In [18]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.prompts import PromptTemplate
import pickle


In [28]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./multivector_chroma_db_001"
)

with open("parent_documents.pkl", "rb") as f:
    parent_documents = pickle.load(f)


# 🔹 Reconstruís el store
store = InMemoryStore()
store.mset([(d.metadata["id"], d) for d in parent_documents])

# 🔹 Reconstruís el retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key="parent_id",
    search_kwargs={"k": 3} # reminder: top 3 similitud para los chunks resumidos, no para los fallos completos
)

In [ ]:
# Define LLM

llm = ChatOpenAI(model='o4-mini')

prompt_template = PromptTemplate.from_template("""
Sos un asistente jurídico. Usá exclusivamente el contenido de los documentos proporcionados para responder la consulta.
No inventes ni infieras información que no esté presente en los textos.
Si no hay jurisprudencia relevante, indicá claramente que no la encontrás.
Tampoco menciones ningún documento si no encontraste juridisprudencia relevante.

Documentos:
{context}

Pregunta:
{question}

Respuesta:
""")

# Cadena que usa esos campos
combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_template)

# Este paso une todo: retriever + documents + fields necesarios
retrieval_chain = (
    RunnablePassthrough.assign( # toma el input lo trasnforma en un diccionaio
        question=lambda x: x["input"],  # copia input → question
        context=lambda x: retriever.invoke(x["input"])    #  resultado del retriever.invoke(...), es decir, documentos relevantes.
    ) # el resultado de esto, pasa a 'combine_docs_chain'
    | combine_docs_chain # usa el prompt, y completa 'context' y 'question' para responder la pregunta
)

response = retrieval_chain.invoke({"input": "¿Qué precedentes existen sobre el control de razonabilidad de multas administrativas en casos de aseguradoras?"})

print(response)